In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os

In [ ]:
# Constant which defines the minimum number of points that need to be present
THRESHOLD = 1000
las_no = 'SP3479'
las_path = '/Users/kevin/cs236g/' + las_no + '_PC_Coventry.pickle'
pc_data = pd.read_pickle(las_path)
# EPSG:27700
pc_gdf = gpd.GeoDataFrame(pc_data, geometry='footprint_polygon')
pc_gdf

In [ ]:
# Select only houses whose point clouds have at least 1000 points
pc_gdf.num_points.describe()
print(len(pc_gdf.loc[pc_gdf.num_points > THRESHOLD]))
pc_gdf = pc_gdf.loc[pc_gdf.num_points > THRESHOLD]

In [ ]:
def sample_a_thousand_points(row):
    
    number_of_points_available = row['normalized_points'].shape[0] 
    select_random_subsample = np.random.randint(number_of_points_available, size=THRESHOLD)
    subsampled_point_cloud = row['normalized_points'][select_random_subsample,:]
    return subsampled_point_cloud

In [ ]:
pc_gdf['subsampled_point_cloud'] = pc_gdf.apply(sample_a_thousand_points, axis=1)
pc_gdf.head()

In [ ]:
print(f"Check name of active geometry column: {pc_gdf.geometry.name}")
print(f"Check CRS of active geometry column: {pc_gdf.geometry.crs}")
pc_gdf = pc_gdf.to_crs(4326)
print(f"Check name of active geometry column: {pc_gdf.geometry.name}")
print(f"Check CRS of active geometry column: {pc_gdf.geometry.crs}")

In [ ]:
pc_gdf['footprint_centroid'] = pc_gdf['footprint_polygon'].centroid
pc_gdf

In [ ]:
pc_gdf = pc_gdf.reset_index(drop=True)

In [ ]:
for idx, row in pc_gdf.iterrows():
    
    print(idx)
    lon, lat = row['footprint_centroid'].coords.xy
    file_name = str(lon[0]) + ',' + str(lat[0])
    path = os.path.join("/Users/kevin/cs236g/data/Train/", file_name)
    np.save(path, row['subsampled_point_cloud'])